# Gabriel Amaral Fuchs (1521531)

## INF1608 - Análise Numérica - 2016.1



# Introdução à Otimização

## Introdução

    O objetivo desta apresentação é apresentar dois métodos de otimização, o 'Golden Section Search' e 'Powell's Method'.
    
    Simplificando o significado de otimização podemos dar a seguinte definição: Dado uma função F(x) queremos encontrar o x que minimiza ou maximiza o seu valor.
    
    Para os métodos a seguir vamos considerar que as variáveis estão sujeitas às seguintes condições:

1) $$g_i(x) = 0, i = 1,2,...,M$$
2) $$h_j(x)<= 0, j = 1,2,...,N$$

3)
A nova função será:

$$F^*(x) = F(x) + \mu P(x) $$

4)
Onde:

$$P(x) = \sum_{i=1}^M [g_i(x)]^2 + \sum_{j=1}^N {max[0,h_j(x)]}^2$$




# Golden Section Search

    Vamos considerar o problema de minimizar uma função f(x) de uma variável única x com a seguinte restrição, c<= x <=d.

## Bracketing

    Antes de aplicarmos o método de otimização(Golden Section Search), devemos estimar o intervalo em que o ponto de minimização da função se encontra. Esta otimização é um processo chamado 'Bracketing'.
    O processo é simples, começamos com um valor inicial x0 e continuamos calculando a função,enquanto seu valor decresce, para x1, x2, x3, ..., até atingirmos o ponto xn onde f(x) aumenta seu valor pela primeira vez, o ponto de mínimo está no intervalo (xn-2, xn).
    A seguir temos a implementação de um algoritmo para tal processo:

In [48]:
from math import log

def bracket(f,x1,h):
    
    c = 1.618033989
    f1 = f(x1)
    x2 = x1 + h
    f2 = f(x2)
    
    # Determine downhill direction and change sign of h if needed
    if f2 > f1:
    
        h = -h
        x2 = x1 + h; f2 = f(x2)

    # Check if minimum between x1 - h and x1 + h
    if f2 > f1:
        
        return x2,x1 - h
    
    # Search loop
    for i in range (100):

        h = c*h
        x3 = x2 + h; f3 = f(x3)

        if f3 > f2:
            
            return x1,x3
        
        x1 = x2; x2 = x3
        f1 = f2; f2 = f3

    print 'Bracket não encontrou um mínimo'

## Golden Section Search

    A seguir temos um exemplo de implementação do método:

In [49]:
def goldenSearch(f,a,b,tol=1.0e-9):

    nIter = (int)(-2.078087*log(tol/abs(b-a)))
    R = 0.618033989
    C = 1.0 - R

    # First telescoping
    x1 = R*a + C*b
    x2 = C*a + R*b
    f1 = f(x1)
    f2 = f(x2)

    # Main loop
    for i in range(nIter):
        
        if f1 > f2:
        
            a = x1
            x1 = x2
            f1 = f2
            x2 = C*a + R*b
            f2 = f(x2)

        else:

            b = x2
            x2 = x1
            f2 = f1
            x1 = R*a + C*b
            f1 = f(x1)
        
    if f1 < f2:
            
        return x1,f1

    return x2,f2

## Exemplos de aplicação

In [50]:
# Exemplo 1:

def f(x):
    
    mu = 1.0
    c = min(0.0, x)
    
    return 1.6*(x**3) + 3.0*(x**2) - 2.0*x + mu*(c**2)

xStart = 1.0
h = 0.01
x1,x2 = bracket(f,xStart,h)
x,fMin = goldenSearch(f,x1,x2)
print 'x =',x
print 'f(x) =',fMin

x = 0.273494113171
f(x) = -0.28985978555


In [51]:
# Exemplo 2:

def f(y):

    B = 48.0
    H = 60.0
    a = B*(H - y)/H
    b = (B - a)/2.0
    A = (B + a)*y/2.0
    Q = (a*y**2)/2.0 + (b*y**2)/3.0
    d = Q/A
    c = y - d
    I = (a*y**3)/3.0 + (b*y**3)/6.0
    Ibar = I - A*d**2

    return -Ibar/c

yStart = 60.0 # Starting value of y
h = 1.0 # Size of first step used in bracketing
a,b = bracket(f,yStart,h)
yOpt,fOpt = goldenSearch(f,a,b)
print 'Optimal y =',yOpt
print 'Optimal S =',-fOpt
print 'S of triangle =',-f(60.0)

Optimal y = 52.1762738706
Optimal S = 7864.43094136
S of triangle = 7200.0


# Powell's Method

$$v(i+1) = v(i) + h \bigtriangledown(f)$$

    

In [53]:
from numpy import identity,array,dot,zeros,argmax
from math import sqrt

def powell(F,x,h=0.1,tol=1.0e-6):

    def f(s): return F(x + s*v) # F in direction of v

    n = len(x) # Number of design variables
    df = zeros(n) # Decreases of F stored here
    u = identity(n) # Vectors v stored here by rows

    for j in range(30): # Allow for 30 cycles:

        xOld = x.copy() # Save starting point
        fOld = F(xOld)
        
        # First n line searches record decreases of F
        for i in range(n):

            v = u[i]
            a,b = bracket(f,0.0,h)
            s,fMin = goldenSearch(f,a,b)
            df[i] = fOld - fMin
            fOld = fMin
            x = x + s*v

        # Last line search in the cycle
        v = x - xOld
        a,b = bracket(f,0.0,h)
        s,fLast = search(f,a,b)
        x = x + s*v

        # Check for convergence
        if sqrt(dot(x-xOld,x-xOld)/n) < tol:
                            
            return x,j+1
        
        # Identify biggest decrease & update search directions
        iMax = argmax(df)
                        
        for i in range(iMax,n-1):

            u[i] = u[i+1]
            u[n-1] = v
                            
    print 'Powell não convergiu'

In [54]:
from numpy import array

def F(x): return 100.0*(x[1] - x[0]**2)**2 + (1 - x[0])**2

xStart = array([-1.0, 1.0])
xMin,nIter = powell(F,xStart)

print 'x =',xMin
print 'F(x) =',F(xMin)
print 'Number of cycles =',nIter

x = [ 1.00000002  1.00000003]
F(x) = 2.89526011325e-16
Number of cycles = 21
